In [1]:
!pip install fuzzywuzzy

In [50]:
import fitz  # PyMuPDF
import os
from fuzzywuzzy import fuzz

def fuzzy_pdf_highlighter(pdf_path, search_string):
    """
    Highlights a sentence based on a fuzzy search across the entire PDF document.

    Parameters:
    pdf_path (str): The path to the PDF document.
    search_string (str): The string to search for in the document.

    Returns:
    str: The path to the modified PDF document.
    """

    doc = fitz.open(pdf_path)
    output_pdf_path = "highlighted_documents"
    if not os.path.exists(output_pdf_path):
        os.makedirs(output_pdf_path)

    filename = os.path.basename(pdf_path)
    filepath = os.path.join(output_pdf_path, filename)


    full_text = ""
    for page in doc:
        full_text += page.get_text() + " "
    words = full_text.split()

    search_length = len(search_string.split())

    # rolling window
    chunks = [' '.join(words[i:i+search_length]) for i in range(len(words)-search_length+1)]
    match_score, match_chunk = max((fuzz.ratio(search_string, chunk), chunk) for chunk in chunks)
    
    print("Match score:", match_score)
    if match_score > 85:  # Adjust score_cutoff as needed
        print("Match found:", match_chunk)

        for page in doc:
            text_instances = page.search_for(match_chunk)
            for inst in text_instances:
                highlight = page.add_highlight_annot(inst)

    doc.save(filepath, garbage=4, deflate=True)
    doc.close()
    return filepath

# Example usage
pdf_path = "text1.PDF"
pdf_path = "test5.pdf"
search_string = """4ej  länder  skog78kötsefff    ln  å  allmänningsskogen  till  men  eller 

om
eljest synnerliga skäl föreligga till avyttringen."""

search_string = """Bestämmelserna i 13 och        14 §§ skall dock fortfarande tillämpas i fråga


om mark som införlivats med-4      allmänningsskog genom förordnande som har




meddelats före lagens ikraftträdande."""

search_string = """förfarande enligt tillämplig nationell lag. Det är därför
nödvändigt att begränsa tillämpningen av principen om
likabehandling.
(18) Domstolen har konsekvent hävdat att Barberprotokollet (4)
inte påverkar rätten att ansluta sig till ett företags- eller
yrkesbaserat pensi


onssystem och att begränsningen i tiden
av domens rättsverkan i mål C-262/88 inte gäller i fråga om

rätten att vara ansluten till ett yrkespensionssystem.
Domstolen fastställde  även att nationella regler om frister
för väckande av     talan enligt nationell rätt kan åberopas
gentemot arbetstagare som gör gällande sin rätt att vara

"""


fuzzy_pdf_highlighter(pdf_path, search_string)

Match score: 97
Match found: likvärdigt förfarande enligt tillämplig nationell lag. Det är därför nödvändigt att begränsa tillämpningen av principen om likabehandling. (18) Domstolen har konsekvent hävdat att Barberprotokollet (4) inte påverkar rätten att ansluta sig till ett företags- eller yrkesbaserat pensionssystem och att begränsningen i tiden av domens rättsverkan i mål C-262/88 inte gäller i fråga om rätten att vara ansluten till ett yrkespensionssystem. Domstolen fastställde även att nationella regler om frister för väckande av talan enligt nationell rätt kan åberopas gentemot arbetstagare som gör gällande sin rätt att vara


'highlighted_documents/test5.pdf'